In [7]:
import os
os.chdir('../rotaryWing/')

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
from tools.BET import BETheory
from tools.BEM import BEMTheory
from tools.airfoil import airfoil

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
roAir = 1.225   # kg/m3 density of air
roWater = 1000  # kg/m3
g = 9.8         # m/s2

# The airfoil : NACA 0024.
airfoil = airfoil('./airfoil_data/xf-naca0024-il-500000.csv')
lift_slope = airfoil.a()
rBlade = 1 # m  Radius of the blade(rough estimate)
R_co = 0.2 # mine was 0.15
omega_air = 650*2*np.pi/60 
omega_water = 25*2*np.pi/60
V = 0

massOfStrutcutre= 10# kg this mass of everything other than payload mass.
# Given
massOfPayload = 5
volOfPayload = 0.0025
b = 3       # no of Blades . Tentatively decided on three
areaBlade = np.pi*(rBlade**2)
total_vol = 0.008 # The payload is gonna be taking a place inside this volume


In [10]:
def vel_power(V, medium='air'):
    if medium == 'air':
        T = (massOfStrutcutre + massOfPayload)*g; ro = roAir
    elif medium == 'water':
        ro = roWater
        T = (massOfStrutcutre + massOfPayload)*g - total_vol*ro*g
        
    ind_vel_MT = (-1*V/2) + np.sqrt(((V/2)**2) + (T/(2*ro*areaBlade)))
    power = T*(V + ind_vel_MT)
    return round(ind_vel_MT, 5) , round(power,4)
idealP_air = vel_power(V, 'air')[1]
ideaP_water = vel_power(V,'water')[1]
print(idealP_air, ideaP_water)

642.4185 7.168


In [11]:
collective_pitch_air = 6*np.pi/180
collective_pitch_water = 5*np.pi/180
def linear_taper(r_ratio):
    return 0.15 - 0.02*r_ratio*rBlade
def linear_twist_air(r_ratio):
    return collective_pitch_air
def linear_twist_water(r_ratio):
    return collective_pitch_water
def drag_air(AoA):
    return 0.013
def drag_water(AoA):
    return 0.15
radii = np.linspace(R_co, rBlade, 100)
c_r = [linear_taper(r/rBlade) for r in radii ]
c_mean = np.mean(c_r)

In [12]:
rotor_air = BEMTheory(
                angular_vel = omega_air, no_of_blades= b, 
                radius= rBlade, lift_slope=lift_slope, drag = drag_air, linear_twist=linear_twist_air, 
                climb_vel=V , root_cutouts=R_co, linear_taper=linear_taper)
rotor_water = BEMTheory(
                angular_vel = omega_water, no_of_blades= b, 
                radius= rBlade, lift_slope=lift_slope, drag = drag_water, linear_twist=linear_twist_water, 
                climb_vel=V , root_cutouts=R_co, linear_taper=linear_taper, medium='water')

In [13]:
print(f'Air : T,P = {rotor_air.T():.4f}, {rotor_air.P():.4f}')
print(f'Water : T,P = {rotor_water.T():.4f}, {rotor_water.P():.4f}')

Air : T,P = 167.4234, 428.9485
Water : T,P = 69.1585, 143.1832


In [14]:
for r_ratio in [0.18,0.3,0.4,0.45,0.5,0.6,0.68,0.75,0.8,0.9,0.95,0.98]:
    print(round(rotor_air.AoA(r_ratio)*180/np.pi,2), '-'*3,  round(rotor_water.AoA(r_ratio)*180/np.pi,2))

3.74 --- 0.57
4.17 --- 0.84
4.39 --- 1.03
4.47 --- 1.12
4.54 --- 1.2
4.66 --- 1.34
4.74 --- 1.45
4.8 --- 1.54
4.84 --- 1.6
4.9 --- 1.76
4.93 --- 2.0
4.96 --- 2.44


In [15]:
#Reynolds Number
kinematic_viscosity_water = 1.267e-6
kinematic_viscosity_air   = 1.511e-5
re_air = 5*c_mean/kinematic_viscosity_air
re_water = 5*c_mean/kinematic_viscosity_water
print(f'Reynolds Number Air: {re_air:1f}, Water: {re_water:1f}')

Reynolds Number Air: 45665.122435, Water: 544593.528019
